<a href="https://colab.research.google.com/github/limshaocong/analyticsEdge/blob/main/Zero_shot_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-ffbb39df-4dea-15a4-c01e-6e3d900d1a0f)


In [3]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 15.0 MB/s 
     |████████████████████████████████| 895 kB 73.4 MB/s 
     |████████████████████████████████| 3.0 MB 60.2 MB/s 
     |████████████████████████████████| 1.2 MB 62.1 MB/s 


In [4]:
from transformers import pipeline
import pandas as pd

In [5]:
classifier = pipeline("zero-shot-classification", # transfer learning
                      model = "joeddav/xlm-roberta-large-xnli", # multi-lingual model
                      device = 0) # to utilize GPU

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [6]:
import re
from datetime import datetime
import os.path
from os import path
import requests

In [18]:
def analyse(ticker):
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  df = pd.read_csv(file_path)
  
  df['Text_clean'] = df['Text_clean'].astype(str)

  texts = df["Text_clean"]

  chunks = [texts[x : x + 100] for x in range(0, len(texts), 100)] # split into chunks

  results_list = []

  for idx in range(len(chunks)):

    cp1 = datetime.now()
    
    chunk = chunks[idx].tolist()

    candidate_labels = ["positive", "negative", "neutral"]

    hypothesis_template = "The sentiment of this tweet is {}."

    #dicts = classifier(text, candidate_labels, hypothesis_template = hypothesis_template)
    #result = dicts["labels"][0]
    #results_list.append(result)
    
    list_results_dicts = classifier(chunk, candidate_labels, hypothesis_template = hypothesis_template)

    if type(list_results_dicts) != dict:
      new_list = [i["labels"][0] for i in list_results_dicts] # not robust when chunk size == 1
      results_list.extend(new_list)
    
    else:
      new_list = list_results_dicts["labels"][0]
      results_list.append(new_list)
    
    #results_list.extend(new_list)

    cp2 = datetime.now()

    print(str(idx + 1) + str("/") + str(len(chunks)) + str(" --- ") + str(cp2 - cp1))

  return results_list

In [8]:
from google.colab import files

ticker_path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/Tickers_reduced_full.csv"
tickers = pd.read_csv(ticker_path)
tickers_list = tickers["Tickers"].tolist()
tickers_list = tickers_list[126:]

path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main//Datasets/Tweet_Analysis/"

In [9]:
tickers_list[:20]

['UGI', 'UNIT', 'VC', 'VERI', 'VIAC', 'VZ', 'WELL', 'WFC', 'WMT', 'Z', 'ZG']

In [ ]:
"""
for ticker in tickers_list:
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  response = requests.get(file_path)

  if response.status_code < 400:
 
    print(ticker + str(" is present"))

    file_path = path + ticker + str("_tweets_2020_cleaned.csv")

    print(file_path)
  
    df = pd.read_csv(file_path)

    results_list = analyse(ticker)

    df["Sentiment"] = results_list

    newfilename = ticker + str("_tweets_2020_tagged.csv")

    df.to_csv(newfilename, index = False)
  
  else:

    print(ticker + str(" is missing"))

    skipped.append(ticker)

    print(skipped)

    continue

"""

In [10]:
skipped = ['CAT',
 'CMCSA',
 'CORE',
 'CRDA',
 'CSCO',
 'CUBI',
 'DISCK',
 'DUK',
 'EBAY',
 'EPR',
 'F',
 'FCNCA',
 'FNB',
 'FOX',
 'FOXA',
 'FWONA',
 'FWONK',
 'MA',
 'MDP',
 'MDT',
 'MMM',
 'MRK',
 'MS',
 'NEE',
 'NKE',
 'NWS',
 'NWSA',
 'OKE',
 'PENN',
 'PFE',
 'PK',
 'POST',
 'PRGO',
 'PRU',
 'QADA',
 'QCOM',
 'QRTEA',
 'RMD',
 'ROG',
 'SBRA',
 'SBUX',
 'SF',
 'SITC',
 'SLM',
 'STAG',
 'STT',
 'STZ',
 'SWM',
 'SYY',
 'T',
 'TAP',
 'TPB',
 'TSLA',
 'TWTR',
 'UA',
 'UAA',
 'AMZN',
 'GOOG',
 'GOOGL',
 'NFLX',
 'IBM',
 'BA',
 'FB']

In [26]:
skipped_1 = ['EBAY', 'FOX']
skipped_2 = []
skipped_3 = []
skipped_4 = []
skipped_5 = {'AAPL',
 'ASB',
 'ASH',
 'FWONA',
 'FWONK',
 'HY',
 'PFE',
 'TSLA',
 'WELL'}

In [27]:
complete = []

for ticker in skipped_5:
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  response = requests.get(file_path)

  if response.status_code < 400:
      
    print(ticker + str(" is present"))

    file_path = path + ticker + str("_tweets_2020_cleaned.csv")

    print(file_path)
  
    df = pd.read_csv(file_path)

    results_list = analyse(ticker)

    df["Sentiment"] = results_list

    newfilename = ticker + str("_tweets_2020_tagged.csv")

    df.to_csv(newfilename, index = False)

    complete.append(ticker)
  
  else:

    print(ticker + str(" is missing"))

    continue

Streaming output truncated to the last 5000 lines.
866/4754 --- 0:00:01.507448
867/4754 --- 0:00:01.644978
868/4754 --- 0:00:01.498874
869/4754 --- 0:00:01.600985
870/4754 --- 0:00:01.597155
871/4754 --- 0:00:01.627197
872/4754 --- 0:00:01.725006
873/4754 --- 0:00:01.730671
874/4754 --- 0:00:01.661496
875/4754 --- 0:00:01.652101
876/4754 --- 0:00:01.544602
877/4754 --- 0:00:01.502947
878/4754 --- 0:00:01.480086
879/4754 --- 0:00:01.411763
880/4754 --- 0:00:01.660484
881/4754 --- 0:00:01.624838
882/4754 --- 0:00:01.622512
883/4754 --- 0:00:01.716870
884/4754 --- 0:00:01.629131
885/4754 --- 0:00:01.777648
886/4754 --- 0:00:01.639875
887/4754 --- 0:00:01.728825
888/4754 --- 0:00:01.605579
889/4754 --- 0:00:01.612835
890/4754 --- 0:00:01.623553
891/4754 --- 0:00:01.510591
892/4754 --- 0:00:01.543743
893/4754 --- 0:00:01.595610
894/4754 --- 0:00:01.522338
895/4754 --- 0:00:01.612052
896/4754 --- 0:00:01.528296
897/4754 --- 0:00:01.607345
898/4754 --- 0:00:01.503384
899/4754 --- 0:00:01.6611